In [1]:
import pandas as pd

In [2]:
flow_df = pd.read_csv('cleaning1/1160/flow.tsv', delimiter='\t', header=None)
speed_df = pd.read_csv('cleaning1/1160/speed.tsv', delimiter='\t', header=None)
occupancy_df = pd.read_csv('cleaning1/1160/occupancy.tsv', delimiter='\t', header=None)

In [40]:
df = pd.DataFrame({'A' : []})
del df['A']
df

""


In [41]:
seriesList = []
for col in flow_df:
    df[col] = pd.Series(data = list(zip(flow_df[col],speed_df[col], occupancy_df[col])))

In [42]:
df

,0,1,2
0,"(8, 68.0, 3.0)","(13, 67.0, 4.0)","(255, 0.0, 4.0)"
1,"(9, 67.0, 4.0)","(4, 60.0, 2.0)","(255, 0.0, 4.0)"
2,"(11, 66.0, 4.0)","(10, 64.0, 5.0)","(255, 0.0, 4.0)"
3,"(-8, 71.0, 3.0)","(3, 63.0, 1.0)","(255, 0.0, 4.0)"
4,"(5, 65.0, 2.0)","(9, 70.0, 3.0)","(255, 0.0, 4.0)"
5,"(7, 66.0, 2.0)","(10, 64.0, 4.0)","(255, 0.0, 4.0)"
6,"(6, 66.0, 2.0)","(6, 67.0, 3.0)","(255, 0.0, 4.0)"
7,"(0, 65.0, 0.0)","(3, 67.0, 2.0)","(255, 0.0, 4.0)"
8,"(10, 68.0, 4.0)","(8, 74.0, 3.0)","(255, 0.0, 4.0)"
9,"(7, 79.0, 3.0)","(3, 61.0, 2.0)","(255, 0.0, 4.0)"


In [44]:
def containsNeg(vec):
    if(vec[0] < 0):
        return 1
    elif(vec[1] < 0):
        return 1
    elif(vec[2] < 0):
        return 1
    else:
        return 0

In [57]:
containsNeg(df[0][3])

1